# Hackathon Dotz

##### Classificar as observações por (SUB-CATEGORIA | CATEGORIA | DEPARTAMENTO)

In [16]:
# Importação dos pacotes para Analise

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
from google.colab import files  
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
import nltk
from nltk import word_tokenize 
nltk.download('punkt')
import collections

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
#Carregar os dados para o google colab

uploaded = files.upload()

Saving Dados Gerais Datasets.txt to Dados Gerais Datasets.txt


In [0]:
# Importação dos dataset

df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv('Hackathon_Base_Teste.csv')

### Análise dos dados

In [19]:
# Exemplos dos dados treino

df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA


In [20]:
# Exemplos dos dados treino teste

df2.head(2)

,0,DESCRIÇÃO PARCEIRO
0,1,SAL ROSA HIMALAIA C/ ALHO 500G
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952


In [21]:
# Verificação de nulos

df.isna().sum()

DESCRIÇÃO PARCEIRO    0
SUB-CATEGORIA         0
CATEGORIA             0
DEPARTAMENTO          0
dtype: int64

In [22]:
# Tamanho do dataset de treino (Vamos usar para treino e teste afim de validar os dados)

df.shape

(22009, 4)

In [23]:
# Quantidade das SUB-CATEGORIA, das CATEGORIA e das DEPARTAMENTO

classifica = df[['DEPARTAMENTO','CATEGORIA', 'SUB-CATEGORIA']]
classifica.nunique()

DEPARTAMENTO       53
CATEGORIA         332
SUB-CATEGORIA    1968
dtype: int64

In [24]:
# Tamanho das descrições

np.sort(df['DESCRIÇÃO PARCEIRO'].str.len().unique())

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 63, 64, 65, 67, 69, 70, 71, 74])

In [25]:
# Tamanho médio das descrições

df['DESCRIÇÃO PARCEIRO'].str.len().unique().mean()

37.292307692307695

In [26]:
#texto = df[['DESCRIÇÃO PARCEIRO']]
texto = df[['DESCRIÇÃO PARCEIRO', 'DEPARTAMENTO']].query('DEPARTAMENTO in ("CUIDADOS COM O BANHEIRO","GURDUROSOS","DECORAÇÃO")')
texto['TOKENIZED'] = texto['DESCRIÇÃO PARCEIRO'].apply(word_tokenize)
texto = list(texto.TOKENIZED)
lista = [item for sublista in texto for item in sublista]
lista = sorted(lista)
unicas = set(lista)
print('Quantidade de observações:', len(texto), '\n')
print('Quantidade de GERAL palavras:', len(lista), '\n')
print('Quantidade de UNICA palavras:', len(unicas), '\n')
counter = collections.Counter(lista)
arquivo = open('dotz.txt', 'w')
arquivo.write(str(counter))
arquivo.close()

Quantidade de observações: 489 

Quantidade de GERAL palavras: 2925 

Quantidade de UNICA palavras: 984 



### Etapa de Preparação do Dados

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)
df['DESC_AJUSTADA_SUB'] = df['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)
# Dados de Teste
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9].+', '', regex=True)
df2['DESC_AJUSTADA_SUB'] = df2['DESCRIÇÃO PARCEIRO'].str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
pontuacao=['.',',','-','+',':',';','&','+','/','!','#','%','(',')','  ']
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')
  df['DESC_AJUSTADA_SUB'] = df['DESC_AJUSTADA_SUB'].str.replace(x, ' ')
  # Dados de Teste
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')
  df2['DESC_AJUSTADA_SUB'] = df2['DESC_AJUSTADA_SUB'].str.replace(x, ' ')

In [28]:
df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA,DESC_AJUSTADA_SUB
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,PASTA INT VITAPOWER,PASTA INT VITAPOWER KG AMEND SHOT
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,ESPONJA BETTANIN BRILHUS C 1,ESPONJA BETTANIN BRILHUS C


In [0]:
# Variáveis importantes
# Palavras para retirar da análise
stop_words=['de','do','da','kg','g','l','ml','e','gfa','un','lt','n','la','daiso','acasa']
# Tamanho da validação de teste
test_size= 0.01
# Interações no modelo
max_iter = 10000
validation_fraction = 0.0001
fit_intercept = True

### Etapa 1 - Classificando DEPARTAMENTO

In [0]:
#Criando a nova para classificar DEPARTAMENTO
base = df

# Selecionando apenas o item a ser classificado e o target do DF principal
base_data = base['DESC_AJUSTADA']
base_target = base['DEPARTAMENTO']

# Transformando a descrição em vetor
vetor = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii')
vetor_x = vetor.fit_transform(base_data)

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(vetor_x, base_target, test_size=test_size, random_state=27)

# Aqui nós configuramos o modelo para treinar o classificador
clf = PassiveAggressiveClassifier(max_iter=max_iter, validation_fraction=validation_fraction, fit_intercept=fit_intercept)

In [31]:
# Mostrar o tempo de processamento
%%time

# Executar o treinamento
clf.fit(x_train, y_train)

CPU times: user 1.19 s, sys: 802 ms, total: 2 s
Wall time: 1.04 s


PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=10000, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.0001, verbose=0,
                            warm_start=False)

In [32]:
# Avaliando a performance com predição
predicted = clf.predict(x_test)

print('\n###Indicadores Classificação###\n')
print(metrics.classification_report(y_test, predicted))


###Indicadores Classificação###

                          precision    recall  f1-score   support

              AUTOMOTIVO       1.00      0.67      0.80         3
                 AÇOUGUE       1.00      1.00      1.00         3
      BEBIDAS ALCOÓLICAS       1.00      1.00      1.00        19
  BEBIDAS NÃO ALCOÓLICAS       1.00      0.91      0.95        11
               BRINQUEDO       1.00      1.00      1.00         1
                 CALÇADO       1.00      1.00      1.00         5
      CAMA, MESA E BANHO       1.00      1.00      1.00         2
     CONGELADOS SALGADOS       1.00      1.00      1.00         2
     CUIDADOS COM A CASA       1.00      1.00      1.00         1
  CUIDADOS COM A COZINHA       1.00      1.00      1.00         2
    CUIDADOS COM A ROUPA       1.00      1.00      1.00         5
 CUIDADOS COM MÃOS E PÉS       1.00      1.00      1.00         4
 CUIDADOS COM O BANHEIRO       0.67      1.00      0.80         2
   CUIDADOS COM O CABELO       1.00      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
# Predição em novos dados com o DF  de teste
base_novos = df2['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n = vetor.transform(base_novos)

#Previsão dos novos dados
predicted_novos = clf.predict(vetor_n)

# Adicionando a coluna para o novo DF
df_dep = df2[['DESCRIÇÃO PARCEIRO','DESC_AJUSTADA']]
df_dep['DEPARTAMENTO'] = predicted_novos
df_dep['DESC_AJUSTADA_SUB'] = df2['DESC_AJUSTADA_SUB']

# Novos Dados de teste incluindo o departamento
df_dep.head(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R


### Etapa 2 - Classificando CATEGORIA

In [0]:
# Criando a nova base para classificar CATEGORIA
base2 = df

# Selecionando apenas o item a ser classificado e o target do DF principal
base_data2 = base2['DEPARTAMENTO']+' '+base2['DESC_AJUSTADA']
base_target2 = base2['CATEGORIA']

# Transformando a descrição em vetor
vetor2 = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii')
vetor_x2 = vetor2.fit_transform(base_data2)

# Dividindo dataset em treino e teste
x_train2, x_test2, y_train2, y_test2 = train_test_split(vetor_x2, base_target2, test_size=test_size, random_state=27)

In [35]:
# Mostrar o tempo de processamento
%%time

# Aqui nós configuramos o modelo para treinar o classificador
clf2 = PassiveAggressiveClassifier(max_iter=max_iter)

# Executar o treinamento
clf2.fit(x_train2, y_train2)

CPU times: user 7.09 s, sys: 4.96 s, total: 12 s
Wall time: 6.13 s


In [36]:
# Avaliando a performance com predição
predicted2 = clf2.predict(x_test2)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test2, predicted2))


###Indicadores Classificação Categoria###

                                   precision    recall  f1-score   support

                       ABSORVENTE       1.00      1.00      1.00         2
                ACESSÓRIOS ELETRO       1.00      1.00      1.00         2
              ACESSÓRIOS MÃO E PÉ       1.00      1.00      1.00         1
           ACESSÓRIOS PARA CABELO       1.00      1.00      1.00         4
               ALIMENTO PARA CÃES       1.00      1.00      1.00         2
              ALIMENTO PARA GATOS       1.00      1.00      1.00         1
                        ALMOFADAS       1.00      1.00      1.00         1
                        ALVEJANTE       1.00      1.00      1.00         1
        ANTISSÉPTICO BUCAL ADULTO       1.00      1.00      1.00         1
                            ARROZ       1.00      1.00      1.00         1
              ARTIGOS PARA FESTAS       1.00      1.00      1.00         8
               AUTOMOTIVO LIMPEZA       0.50      1.00 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
# Predição em novos dados utilizando o df_dep
base_novos2 = df_dep['DEPARTAMENTO']+' '+ df_dep['DESC_AJUSTADA']

# Tranformando descrição em vetor
vetor_n2 = vetor2.transform(base_novos2)

#Previsão dos novos dados
predicted_novos2 = clf2.predict(vetor_n2)

# Adicionando a coluna no novo DF
df_cat = df_dep
df_cat['CATEGORIA'] = predicted_novos2
df_cat['DESC_AJUSTADA_SUB'] = df_dep['DESC_AJUSTADA_SUB']

# Novos Dados de teste incluindo o departamento
df_cat.head(2)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB,CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G,SAL
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R,ENFEITES DE NATAL


### Etapa 3 - Classificando SUB-CATEGORIA

In [0]:
# Criando a nova para classificar CATEGORIA retirando apenas números
base3 = df

# Selecionando apenas o item a ser classificado e o target do DF principal
base_data3 = base3['DEPARTAMENTO']+' '+base3['CATEGORIA']+' '+base3['DESC_AJUSTADA_SUB']
base_target3 = base3['SUB-CATEGORIA']

# Transformando a descrição em vetor
vetor3 = TfidfVectorizer(stop_words=stop_words, strip_accents='ascii')
vetor_x3 = vetor3.fit_transform(base_data3)

# Dividindo dataset em treino e teste
x_train3, x_test3, y_train3, y_test3 = train_test_split(vetor_x3, base_target3, test_size=test_size, random_state=27)

In [48]:
# Mostrar o tempo de processamento
%%time

# Aqui nós treinamos o classificador
clf3 = PassiveAggressiveClassifier(max_iter=max_iter)

# Executar o treinamento
clf3.fit(x_train3, y_train3)

CPU times: user 43.4 s, sys: 30 s, total: 1min 13s
Wall time: 37.1 s


In [49]:
# Avaliando a performance com predição
predicted3 = clf3.predict(x_test3)

print('\n###Indicadores Classificação Categoria###\n')
print(metrics.classification_report(y_test3, predicted3))


###Indicadores Classificação Categoria###

                             precision    recall  f1-score   support

         ACESSÓRIOS CELULAR       0.00      0.00      0.00         0
       ACESSÓRIOS MANGUEIRA       1.00      1.00      1.00         1
           ACESÓRIOS PANELA       1.00      1.00      1.00         1
        AEROSOL COM PERFUME       1.00      1.00      1.00         1
                     ALFACE       1.00      1.00      1.00         1
                    ALFAJOR       0.00      0.00      0.00         2
                 ALHO FRITO       0.00      0.00      0.00         0
                ALHO OUTROS       0.00      0.00      0.00         1
                 ALHO PASTA       1.00      0.50      0.67         2
           ALTA PERFORMANCE       1.00      1.00      1.00         1
            ANTI-BACTERIANO       1.00      1.00      1.00         1
                 ANTI-QUEDA       1.00      1.00      1.00         1
   ANTISSÉPTICO BUCAL FRESH       0.00      0.00      0.00

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
# Predição em novos dados
base_novos3 = df_cat['DEPARTAMENTO']+' '+df_cat['CATEGORIA']+' '+df_cat['DESC_AJUSTADA_SUB']

# Tranformando descrição em vetor
vetor_n3 = vetor3.transform(base_novos3)

#Previsão dos novos dados
predicted_novos3 = clf3.predict(vetor_n3)

# Adicionando a coluna
df_sub = df_cat
df_sub['SUB-CATEGORIA'] = predicted_novos3

# Novos Dados de teste incluindo o departamento
df_sub.head(10)

,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,DESC_AJUSTADA_SUB,CATEGORIA,SUB-CATEGORIA
0,SAL ROSA HIMALAIA C/ ALHO 500G,SAL ROSA HIMALAIA C ALHO,MERCEARIA SALGADA,SAL ROSA HIMALAIA C ALHO G,SAL,SAL DO HIMALAIA
1,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,JG BOLA NATAL ACASA C,DECORAÇÃO,JG BOLA NATAL ACASA C DR PR BC R,ENFEITES DE NATAL,BOLA DE NATAL
2,MIST PREP LIQ CHA VDE LINEA 56ML-CX LIMAO,MIST PREP LIQ CHA VDE LINEA,MERCEARIA DOCE,MIST PREP LIQ CHA VDE LINEA ML CX LIMAO,FARINÁCEO,MISTURAS
3,ALICATE CUT+ESPAT MUNDIAL SANDY AMARELO,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,CUIDADOS COM MÃOS E PÉS,ALICATE CUT ESPAT MUNDIAL SANDY AMARELO,ACESSÓRIOS MÃO E PÉ,ALICATE
4,MEIA PIZZA PEPPERONI UN,MEIA PIZZA PEPPERONI UN,PRATOS PRONTOS,MEIA PIZZA PEPPERONI UN,PIZZA,PEPPERONI
5,JARRA ACASA VIDRO CURVES OJG378,JARRA ACASA VIDRO CURVES OJG,UTILIDADES DE COZINHA,JARRA ACASA VIDRO CURVES OJG,UTENSÍLIOS DE VIDRO,JARRA/GARRAFA
6,FILME DE PVC CONSERV PACK 30M,FILME DE PVC CONSERV PACK,DESCARTÁVEIS,FILME DE PVC CONSERV PACK M,FILME PVC,FILME PVC
7,CESTO VIME CRAFTWORK S-25X15XH13,CESTO VIME CRAFTWORK S,LAVANDERIA,CESTO VIME CRAFTWORK S XXH,ORGANIZADOR,CESTO DE ROUPA
8,MAC BORTOLINI 500GR OVOS ESPAGUETE .,MAC BORTOLINI,MERCEARIA SALGADA,MAC BORTOLINI GR OVOS ESPAGUETE,MASSA SECA,ESPAGUETE
9,COND NOVEX OLEO OJON 300ML,COND NOVEX OLEO OJON,CUIDADOS COM O CABELO,COND NOVEX OLEO OJON ML,CONDICIONADOR,TRADICIONAL


In [0]:
dotz = df_sub[['DESCRIÇÃO PARCEIRO', 'SUB-CATEGORIA', 'CATEGORIA']]
dotz.to_csv('dotz.csv')

In [43]:
from joblib import dump, load
dump(clf, 'modelo_etapa1.joblib') 
dump(clf2, 'modelo_etapa2.joblib') 
dump(clf3, 'modelo_etapa3.joblib') 
#clf = load('modelo_etapas.joblib')

['modelo_etapa3.joblib']

### Referências

https://medium.com/luisfredgs/classificando-textos-com-machine-learning-e054ca7bf4e0

https://www.alura.com.br/artigos/classificando-textos-com-python